## Library

In [1]:
import os
import sys
import time
import copy
import random

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt

%matplotlib inline

## Hyper-parameters

In [2]:
learning_rate = 1e-3
batch_size = 100
num_epochs = 100
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Make Dataset, DataLoader

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

In [4]:
train_data_all = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

n_val = int(len(train_data_all) * 0.2)
n_train = len(train_data_all) - n_val

train_data, valid_data = random_split(train_data_all, [n_train, n_val], generator=torch.Generator().manual_seed(0))

train_loader = DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2
)

valid_loader = DataLoader(
    valid_data,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2
)

Files already downloaded and verified


## Define Model

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer_1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),  # (32, 32, 32)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) # (16, 16, 32)
        )

        self.layer_2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),  # (16, 16, 64)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # (8, 8, 64)
        )

        self.layer_3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),  # (8, 8, 128)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) # (4, 4, 128)
        )
        
        self.fc_1 = nn.Linear(4*4*128, 64)
        self.fc_2 = nn.Linear(64, 10)


    def forward(self, x):
        x = self.layer_1(x)
        x = self.layer_2(x)
        x = self.layer_3(x)

        x = x.view(x.size(0), -1) # flatten
        x = self.fc_1(x)
        x = self.fc_2(x)

        return x

## Training

In [6]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

In [7]:
best_model_weights = copy.deepcopy(model.state_dict())
early_stop = 5
stop_count = 0
best_acc = 0.0

dataloaders = {'train': train_loader, 'val': valid_loader}

for epoch in range(num_epochs):
    print()
    print(f'[Epoch {epoch+1}/{num_epochs}]')

    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        step_loss = 0.0
        correct_count = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            # train일 경우 loss, optimizer를 업데이트
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                preds = torch.argmax(outputs, dim=-1)
                loss = criterion(outputs, labels)
            
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            step_loss += loss.item() * inputs.size(0)
            correct_count += torch.sum(preds == labels)
        
        epoch_loss = step_loss / len(dataloaders[phase].dataset)
        epoch_acc = correct_count.double() / len(dataloaders[phase].dataset)

        print(f"{phase} Loss: {epoch_loss:.3f} Acc: {epoch_acc:.3f}")

        if phase == 'val' and best_acc < epoch_acc:
            best_acc = epoch_acc
            best_model_weights = copy.deepcopy(model.state_dict())
            stop_count = 0
            print('...best weigths update!...')

        elif phase == 'val' and best_acc >= epoch_acc:
            stop_count += 1

    if early_stop == stop_count:
        print()
        print("Early Stopping...")
        break

print(f"Best validation Accuracy: {100*best_acc:.3f}%")
model.load_state_dict(best_model_weights)


[Epoch 1/100]
train Loss: 1.394 Acc: 0.498
val Loss: 1.114 Acc: 0.603
...best weigths update!...

[Epoch 2/100]
train Loss: 0.974 Acc: 0.657
val Loss: 0.943 Acc: 0.674
...best weigths update!...

[Epoch 3/100]
train Loss: 0.798 Acc: 0.724
val Loss: 0.870 Acc: 0.703
...best weigths update!...

[Epoch 4/100]
train Loss: 0.686 Acc: 0.761
val Loss: 0.812 Acc: 0.723
...best weigths update!...

[Epoch 5/100]
train Loss: 0.597 Acc: 0.792
val Loss: 0.827 Acc: 0.724
...best weigths update!...

[Epoch 6/100]
train Loss: 0.521 Acc: 0.818
val Loss: 0.770 Acc: 0.749
...best weigths update!...

[Epoch 7/100]
train Loss: 0.450 Acc: 0.842
val Loss: 0.792 Acc: 0.749

[Epoch 8/100]
train Loss: 0.387 Acc: 0.864
val Loss: 0.816 Acc: 0.750
...best weigths update!...

[Epoch 9/100]
train Loss: 0.331 Acc: 0.885
val Loss: 0.840 Acc: 0.748

[Epoch 10/100]
train Loss: 0.281 Acc: 0.901
val Loss: 0.925 Acc: 0.745

[Epoch 11/100]
train Loss: 0.240 Acc: 0.913
val Loss: 1.005 Acc: 0.737

[Epoch 12/100]
train Loss: 

<All keys matched successfully>

## Load Test Data

In [8]:
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2
)

Files already downloaded and verified


## Test

In [9]:
# Entire accuracy
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, pred = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (pred == labels).sum().item()

print(f'Accuracy: {100*correct / total:.3f}%')

Accuracy: 75.400%


In [10]:
# class accuracy
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, pred = torch.max(outputs, 1)

        for label, prediction in zip(labels, pred):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

for classname, correct_count in correct_pred.items():
    class_accuracy = 100 * correct_count / total_pred[classname]
    print(f"Accuracy for class {classname} is: {class_accuracy:.3f}%")

Accuracy for class plane is: 78.800%
Accuracy for class car is: 89.400%
Accuracy for class bird is: 70.000%
Accuracy for class cat is: 57.100%
Accuracy for class deer is: 77.600%
Accuracy for class dog is: 58.100%
Accuracy for class frog is: 78.100%
Accuracy for class horse is: 77.400%
Accuracy for class ship is: 85.100%
Accuracy for class truck is: 82.400%
